<a href="https://colab.research.google.com/github/CarlosPascoli/alura_space/blob/master/Corretor_Delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install requests

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive, files
import os
import requests

In [3]:
df = pd.read_csv('/content/PUXADA_MES.csv', sep='|', low_memory=False)

# Configurações da PuxadaMES

In [4]:
df = df.rename(columns={'wc': 'WC', 'ns': 'NS'})

df = df.iloc[:, 1:]

# Convert date columns to datetime, handling errors gracefully
date_columns = ['Start Date', 'Actual Start Date', 'Finish Date', 'Actual Finish Date']
for col in date_columns:
    try:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    except (ValueError, TypeError):
        # Handle cases where conversion fails
        df[col] = pd.NaT  # Use pandas' Not-a-Time for invalid dates

# Subtrai 3 horas de cada coluna
df['Start Date'] = df['Start Date'] - pd.Timedelta(hours=3)
df['Actual Start Date'] = df['Actual Start Date'] - pd.Timedelta(hours=3)
df['Finish Date'] = df['Finish Date'] - pd.Timedelta(hours=3)
df['Actual Finish Date'] = df['Actual Finish Date'] - pd.Timedelta(hours=3)

# prompt: Divida a coluna Man Hours por 24
df['Man Hours'] = df['Man Hours'] / 24
df['Actual Hours'] = df['Actual Hours'] / 24

df['Duration'] = df['Man Hours'] * df['Order Quantity']

# Get the current column order
cols = df.columns.tolist()

# Find the index of 'Order Quantity'
order_quantity_index = cols.index('Order Quantity')

# Create a new column order list
new_cols = cols[:order_quantity_index + 1] + ['Duration'] + cols[order_quantity_index + 1:-1]

# Reorder the DataFrame
df_puxadaMES = df[new_cols]

df_puxadaMES['Duration'] = df_puxadaMES['Duration'] * 24

# df_operacoesComplete

In [5]:
# Filter for 'COMPLETE' status
df_operacoesComplete = df_puxadaMES[df_puxadaMES['Oper Status'] == 'COMPLETE']

# Sort by 'Actual Finish Date' in descending order (most recent first)
df_operacoesComplete = df_operacoesComplete.sort_values(by='Actual Finish Date', ascending=False)

In [6]:
df_operacoesComplete.head()

,Shop Order,Part Number,PN Description,Category,Order Status,Operation,Oper Status,WC Name,Oper Description,WC,...,Null4,Start Date,Actual Start Date,Order Quantity,Duration,Man Hours,Actual Hours,Finish Date,Actual Finish Date,Null5
149844,43900890,506-24501-4001,"REFRESHMENT 6 PAX-PBE, G18 ASSY",SPARE PARTS,RUN,695,COMPLETE,GPX - PREPARAÇÃO DE VERNIZ (G18),LIXAMENTO FUNDO E TROCA DE ISOLAMENTO,GMG0208,...,NaN,2025-02-11 15:30:00,2025-02-03 07:07:24,1,11.6,0.483333,0.0,2025-02-11 15:30:00,2025-02-03 09:53:29,NaN
129399,43880619,546-58032-401-LF034,TRIM & FINISH FWD PARTITION STOWAGE,BASIC MTO,RUN,230,COMPLETE,GPX - PREPARAÇÃO DE VERNIZ (PRAETOR),LIXAM. FUNDO E TROCA DE ISOL.,GMP0208,...,NaN,2025-01-07 06:00:00,2025-02-03 09:08:57,1,8.0,0.333333,0.0,2025-01-08 15:30:00,2025-02-03 09:53:24,NaN
127776,43881399,546-58009-401-LF034,TRIM FINISH CS TABLE SHROUD,BASIC MTO,RUN,110,COMPLETE,GPX - PREPARAÇÃO DE VERNIZ (PRAETOR),LIXAM. 3ºFUNDO E TROCA DE ISOL.,GMP0208,...,NaN,2024-12-02 06:00:00,2025-02-03 09:07:23,1,2.0,0.083333,0.0,2024-12-03 15:30:00,2025-02-03 09:53:24,NaN
127862,43883440,546-58009-401-LF034,TRIM FINISH CS TABLE SHROUD,BASIC MTO,RUN,110,COMPLETE,GPX - PREPARAÇÃO DE VERNIZ (PRAETOR),LIXAM. 3ºFUNDO E TROCA DE ISOL.,GMP0208,...,NaN,2024-12-04 06:00:00,2025-02-03 09:07:23,1,2.0,0.083333,0.0,2024-12-05 15:30:00,2025-02-03 09:53:24,NaN
128230,43880613,546-58040-403-LF034,TRIM & FINISH LH COCKPIT PARTITION,BASIC MTO,RUN,210,COMPLETE,GPX - PREPARAÇÃO DE VERNIZ (PRAETOR),LIXAM. FUNDO E TROCA DE ISOL.,GMP0208,...,NaN,2024-12-16 06:00:00,2025-02-03 09:07:23,1,6.0,0.250000,0.0,2024-12-17 15:30:00,2025-02-03 09:53:24,NaN


# df_deltas

In [7]:
df_deltas = df_puxadaMES[df_puxadaMES['Finish Date'].isnull()]


# df_cd

' Entrando na API
res = requests.get('http://qualcorp.emb:8000/linha').json()
jsonCDs = []

' Passando por todos os dados e filtrando apenas os que desejo usar
for i in range(0, len(res['data'])):
    if res['data'][i]['site'] == "GPX" and res['data'][i]['aResp'] == "Fabrica de Móveis":
        jsonCDs.append(res['data'][i])

' Eliminando os dados desnecessários
for j in range(0, len(jsonCDs)):
    #jsonCDs[j].pop('om')
    jsonCDs[j].pop('agingOmHora')
    jsonCDs[j].pop('agingOmDia')
    jsonCDs[j].pop('agingFluxoOmHora')
    jsonCDs[j].pop('cenTrabalho')
    jsonCDs[j].pop('grupoGeral')
    jsonCDs[j].pop('grupoContaOM')
    jsonCDs[j].pop('cdAberta')
    #df_cd[j].pop('consideraFluxo')

' Lendo dados com pandas
df_cd = pd.DataFrame(jsonCDs)

df_cd = df_cd.rename(
    columns={'priorizacao': 'Priorização', 'nqm': 'Nota', 'op': 'OP', 'aplic': 'Aplic', 'descProb': 'Descrição',
            'quantidadeOM': 'Qtd. OM', 'om': 'OM', 'dataNotaCompleta': 'Data da Emissão', 'agingCdDia': 'Aging da CD (dias)',
            'agingCdHora': 'Aging da CD (h)', 'agingFluxoCdDia': 'Tempo Fluxo CD (dias)',
            'agingFluxoCdHora': 'Tempo Fluxo CD (h)',
            'quantidadeAdendo': 'Qnt de Adendos', 'grupoContaCD': 'Área Prox Ação', 'idProxAcao': 'Prox Ação CD',
            'proxAcao': 'Nome Prox Ação CD', 'site': 'Site', 'aResp': 'Área Responsável', 'prog': 'Programa',
            'ctAtual': 'CT Atual', 'sn': 'SN', 'tipoDefeito': 'Grupo Defeito', 'descTipoDefeito': 'Desc. Defeito',
            'fornecedor': 'Fornecedor', 'descFornecedor': 'Desc. Fornecedor'})


df_cd.to_csv('df_cd.csv', sep='|', index=False)

In [8]:
df_cd = pd.read_csv('/content/df_cd.csv', sep='|', low_memory=False)

In [9]:
df_cd = df_cd[['OP', 'OM']]


In [12]:
df_cd['OM'] = pd.to_numeric(df_cd['OM'], errors='coerce').astype('Int64')


In [14]:
df_cd.dtypes

,0
OP,int64
OM,Int64


# df_OP_OM

In [15]:
# Merge the two dataframes based on the 'Shop Order' and 'OP' columns.
df_opOm = pd.merge(df_deltas, df_cd, left_on='Shop Order', right_on='OM', how='left')

In [16]:
# prompt: Ordenados pela coluna OM

# Sort the merged dataframe by the 'OM' column
df_opOm = df_opOm.sort_values(by='OM')


In [17]:
df_opOm.head()

,Shop Order,Part Number,PN Description,Category,Order Status,Operation,Oper Status,WC Name,Oper Description,WC,...,Actual Start Date,Order Quantity,Duration,Man Hours,Actual Hours,Finish Date,Actual Finish Date,Null5,OP,OM
588,74063477,546-31023-401,,NC REWORK,RUN,111,RELEASED,GPX - APLICAÇÃO DE VERNIZ (PRAETOR),APLIC. VERNIZ 4º FUNDO,GMP0209,...,NaT,1,0.3,0.012500,0.0,NaT,NaT,NaN,43788428.0,74063477
595,74063477,546-31023-401,,NC REWORK,RUN,181,RELEASED,GPX - POLIMENTO (PRAETOR),POLIMENTO DAS PEÇAS - CICLO 1,GMP0210,...,NaT,1,1.0,0.041667,0.0,NaT,NaT,NaN,43788428.0,74063477
594,74063477,546-31023-401,,NC REWORK,RUN,171,RELEASED,GPX - APLICAÇÃO DE VERNIZ (PRAETOR),APLIC. VERNIZ TOP COAT,GMP0209,...,NaT,1,0.3,0.012500,0.0,NaT,NaT,NaN,43788428.0,74063477
582,74063477,546-31023-401,,NC REWORK,RUN,50,COMPLETE,GPX - APLICAÇÃO DE VERNIZ (PRAETOR),APLIC. VERNIZ,GMP0209,...,2025-02-01 23:03:10,1,0.3,0.012500,0.0,NaT,2025-02-02 04:13:32,NaN,43788428.0,74063477
592,74063477,546-31023-401,,NC REWORK,RUN,151,RELEASED,GPX - QUALIDADE FME (PRAETOR),INSPEÇÃO INTERMEDIÁRIA,GMP9999,...,NaT,1,0.2,0.008333,0.0,NaT,NaT,NaN,43788428.0,74063477
